In [ ]:
#####import package ###
import os 
import sys
import numpy as np
import pandas as pd 
import seaborn as sns
from pandas import DataFrame
import sklearn 
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.svm import LinearSVC,SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold,train_test_split,cross_val_score    ###
from sklearn.metrics import confusion_matrix,matthews_corrcoef,make_scorer,roc_curve, auc,roc_auc_score,precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.feature_selection import RFE, RFECV
from sklearn.utils import shuffle  
import warnings
warnings.filterwarnings('ignore')
print(os.getcwd())

In [ ]:
# load dataset
data = pd.read_excel('C:/DM/Diabetes/data.xlsx')


# categorical features:
X_CAT= ['Retinopathy', 'Fatty Liver',  'Hypertension2', 'Lipid Drug', 'Diabetes Drug',   
        'Insulin Use', 'FH of Diabetes', 'FH of Hypertension', 'FH of CAD',  'Sex','Neuropathy', 
        'Diabetic Foot', 'CVD', 'Smoke'
         ]
 
    
# continous features:   
X_CON = ['Diabetes Duration', 'BMI', 'Waist/Hip', 'SBP', 'DBP', 'Na', 'K',
         'Vitamin D', 'FBS', '2HPP', 'HbA1c3', 'Serum Insulin', 'Cholesterol', 'HDL',
         'LDL', 'Triglyceride', 'Uric Acid', 'AST', 'ALT', 'ALKP', 'Microalbumin', 'eGFR', 'Platelet count' 
        ]
         

# split into input and output elements
c = len(data.columns)-1
X = data.iloc[:,1:c]

y= pd.DataFrame(data, index= data.index, columns= ['DN'])

In [ ]:
###############################################################################
#                            Remove Missing values                            #
###############################################################################


#Categorical variables -> Replace Nan with Mode
X[X_CAT]= X[X_CAT].fillna(X[X_CAT].mode().iloc[0])


#Non-Categorical variables -> Replace Nan with Mean
X[X_CON]= X[X_CON].fillna(X[X_CON].mean())

# print total missing
print('X Dimension:', X.shape)
print('X Null Count:', X.isnull().sum())


X[X_CON].describe(include='all')

In [ ]:
###############################################################################
#                          Create train and test set                         #
###############################################################################
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20,
                                                    random_state = 1000)


In [ ]:
###############################################################################
#                         Normalize Continous features                      #
###############################################################################
from sklearn.preprocessing import StandardScaler#, Imputer


scaler = StandardScaler()
X_train[X_CON] = scaler.fit_transform(X_train[X_CON])
X_test[X_CON] = scaler.fit_transform(X_test[X_CON])

In [ ]:
###############################################################################
#                                Classifiers                                #
###############################################################################
# Create list of tuples with classifier label and classifier obj

classifiers = {}
classifiers.update({"XGBOOST": XGBClassifier()})
classifiers.update({"Random Forest": RandomForestClassifier()})
classifiers.update({"LR":LogisticRegression()})
classifiers.update({"SVC": SVC()})
classifiers.update({"DTC": DecisionTreeClassifier()})

parameters = {}
parameters.update({"XGBOOST": { 
                                        "classifier__learning_rate":[0.15,0.1,0.05,0.01,0.005,0.001], 
                                        "classifier__n_estimators": [200],
                                        "classifier__max_depth": [2,3,4,5,6],
                                        "classifier__subsample": [0.8, 0.9, 1]
                                         }})


parameters.update({"Random Forest": { 
                                    "classifier__n_estimators": [200],
                                    "classifier__class_weight": [None, "balanced"],
                                    "classifier__max_features": ["auto", "sqrt", "log2"],
                                    "classifier__max_depth" : [3, 4, 5, 6, 7, 8],
                                    "classifier__min_samples_split": [0.005, 0.01, 0.05, 0.10],
                                    "classifier__min_samples_leaf": [0.005, 0.01, 0.05, 0.10],
                                    "classifier__criterion" :["gini", "entropy"]     ,
                                    "classifier__n_jobs": [-1]
                                     }})


parameters.update({"LR": {
                          "classifier__class_weight": [None, "balanced"],
                          "classifier__n_jobs": [-1],
                          "classifier__C": [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                          "classifier__solver": ["lbfgs", "liblinear", "sag"] 
                          }})



parameters.update({"SVC": { 
                            "classifier__kernel": ["linear"],
                            "classifier__gamma": ["auto"],
                            "classifier__C": [0.1, 0.5, 1, 5, 10, 50, 100],
                            "classifier__degree": [1, 2, 3, 4, 5, 6]
                             }})


parameters.update({"DTC": { 
                            "classifier__criterion" :["gini", "entropy"],
                            "classifier__splitter": ["best", "random"],
                            "classifier__class_weight": [None, "balanced"],
                            "classifier__max_features": ["auto", "sqrt", "log2"],
                            "classifier__max_depth" : [1,2,3, 4, 5, 6, 7, 8],
                            "classifier__min_samples_split": [0.005, 0.01, 0.05, 0.10],
                            "classifier__min_samples_leaf": [0.005, 0.01, 0.05, 0.10],
                             }})


In [ ]:
###############################################################################
#                      Tuning a classifier to use with RFECV                #
###############################################################################
# Define classifier to use as the base of the recursive feature elimination algorithm
selected_classifier = "Random Forest"

classifier = classifiers[selected_classifier]
  
# Define steps in pipeline
steps = [ ("classifier", classifier)]


# Initialize Pipeline object
pipeline = Pipeline(steps = steps)
  
# Define parameter grid
param_grid = parameters[selected_classifier]

# Initialize GridSearch object
gscv = GridSearchCV(pipeline, param_grid, cv = 10,  n_jobs= -1, verbose = 1, scoring = "roc_auc")
                  
# Fit gscv
print(f"Now tuning {selected_classifier}. Go grab a beer or something.")
gscv.fit(X_train, np.ravel(y_train))  

# Get best parameters and score
best_params = gscv.best_params_
best_score = gscv.best_score_
        
# Update classifier parameters
tuned_params = {item[12:]: best_params[item] for item in best_params}
classifier.set_params(**tuned_params)

In [ ]:
# Select Features using RFECV

class PipelineRFE(Pipeline):
    # Source: https://ramhiser.com/post/2018-03-25-feature-selection-with-scikit-learn-pipeline/
    def fit(self, X, y=None, **fit_params):
        super(PipelineRFE, self).fit(X, y, **fit_params)
        self.feature_importances_ = self.steps[-1][-1].feature_importances_
        return self

In [ ]:
###############################################################################
#    Feature Selection: Recursive Feature Selection with Cross Validation   #
###############################################################################
    
# Define pipeline for RFECV

steps = [ ("classifier", classifier)]

pipe = PipelineRFE(steps = steps)

# Initialize RFECV object
feature_selector = RFECV(pipe, cv = 10, step = 1, scoring = "roc_auc", verbose = 1)

# Fit RFECV
feature_selector.fit(X_train, np.ravel(y_train))

# Get selected features
feature_names = X_train.columns
selected_features = feature_names[feature_selector.support_].tolist()


print(selected_features)

In [ ]:
###############################################################################
#                              Performance Curve                           #
###############################################################################
# Get Performance Data
performance_curve = {"Number of Features": list(range(1, len(feature_names) + 1)),
                    "AUC": feature_selector.grid_scores_}
performance_curve = pd.DataFrame(performance_curve)

# Performance vs Number of Features
# Set graph style
sns.set(font_scale = 1.75)
sns.set_style({"axes.facecolor": "1.0", "axes.edgecolor": "0.85", "grid.color": "0.85",
               "grid.linestyle": "-", 'axes.labelcolor': '0.4', "xtick.color": "0.4",
               'ytick.color': '0.4'})
colors = sns.color_palette("RdYlGn", 20)
line_color = colors[3]
marker_colors = colors[-1]

# Plot
f, ax = plt.subplots(figsize=(13, 6.5))
sns.lineplot(x = "Number of Features", y = "AUC", data = performance_curve,
             color = line_color, lw = 4, ax = ax)
sns.regplot(x = performance_curve["Number of Features"], y = performance_curve["AUC"],
            color = marker_colors, fit_reg = False, scatter_kws = {"s": 200}, ax = ax)

# Axes limits
plt.xlim(0.5, len(feature_names)+0.5)
plt.ylim(0.50, 0.90)

# Generate a bolded horizontal line at y = 0
ax.axhline(y = 0.50, color = 'black', linewidth = 1.3, alpha = .7)

# Turn frame off
ax.set_frame_on(False)

# Tight layout
plt.tight_layout()

# Save Figure
plt.savefig("performance_curve.png", dpi = 1080)

pd.set_option('precision', 4)

performance_curve.sort_values('AUC')

In [ ]:
###############################################################################
#                 Feature Selection: Recursive Feature Selection           #
###############################################################################


# Define pipeline for RFECV
#steps = [("scaler", scaler), ("classifier", classifier)]
steps = [ ("classifier", classifier)]

pipe = PipelineRFE(steps = steps)

# Initialize RFE object
feature_selector = RFE(pipe, n_features_to_select = 6, step = 1, verbose = 1)

# Fit RFE
feature_selector.fit(X_train, np.ravel(y_train))

# Get selected features labels
feature_names = X_train.columns
selected_features = feature_names[feature_selector.support_].tolist()
print(selected_features)

In [ ]:
####### list classifiers with best parameters ###

feature = X_train[selected_features]
target = y_train

svm = SVC(probability=True,C=0.1, degree=1, gamma='auto', kernel='linear')

lr = LogisticRegression(C=0.01, n_jobs=-1, solver='sag')

tree = DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=8, max_features='sqrt', min_samples_leaf=0.05,
                       min_samples_split=0.005)


xgboost = xgb.XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=None, max_depth=6,
              min_child_weight=None, #missing=nan, 
                            monotone_constraints=None,
              n_estimators=200, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=0.8, tree_method=None,
              validate_parameters=None, verbosity=None)

rf = RandomForestClassifier(max_depth=8, max_features='log2', min_samples_leaf=0.005,
                       min_samples_split=0.005, n_estimators=200, n_jobs=-1)

classifier_lst = [lr, svm, tree, xgboost, rf]
classifier_lst_name = ['LR', 'SVM', 'DT', 'XGboost', 'RF']

def npv(y_true,y_pred):
    confu = confusion_matrix(y_true,y_pred)
    tn, fp, fn, tp = confu.ravel()
    NPV = tn/(tn+fn)
    return NPV

def spc(y_true,y_pred):
    confu = confusion_matrix(y_true,y_pred)
    #print(confu,confu.sum())
    tn, fp, fn, tp = confu.ravel()
    TNR = tn/(tn+fp)
    #print(TNR)
    return TNR 

def prcauc(model,fea,tar):
    X = np.array(fea)
    y = np.array(tar)
    aucspr = []
    for train, test in cv.split(X, y):
        probas_ = model.fit(X[train], y[train]).predict_proba(X[test])
        precision, recall, thresholds = precision_recall_curve(y[test], probas_[:, 1])
        #precisions.append(interp(mean_recall, precision,recall))
        pr_auc = auc(recall, precision)
        aucspr.append(pr_auc)
    #print(mean_precision)
    prc_auc_CI = np.percentile(aucspr, 2.5), np.percentile(aucspr, 97.5)
    print("prc_auc_CI",prc_auc_CI)
    return prc_auc_CI

npv_score,spc_score,mcc_score = make_scorer(npv),make_scorer(spc),make_scorer(matthews_corrcoef)
prcauc_score = make_scorer(prcauc)


cv = StratifiedKFold(n_splits=5,random_state=1)

In [ ]:
### set StratifiedKFold(CV) =10 
def make_baseline(data,classifier):
    #feature,target = load_data(data)
    classifier.fit(feature,target)
    
    acc = cross_val_score(classifier,feature,target,cv =cv)  ## accuracy 
    acc_mean = np.mean(acc) ###
    
    precisions = cross_val_score(classifier,feature,target,cv =cv,scoring = 'precision')   ### PPV 
    precisions_mean = np.mean(precisions)
        
    print(precisions)
    
    #PRC AUC
    prc_aucs = prcauc(classifier,feature,target)
    print("PRC AUC:",prc_aucs)
    
    f1s = cross_val_score(classifier,feature,target,cv=cv,scoring='f1')
    f1s_mean = np.mean(f1s)
      
    aucs = cross_val_score(classifier,feature,target,cv=cv,scoring='roc_auc') 
    auc_mean = np.mean(aucs)
    
    prc_auc_CI = np.percentile(prc_aucs, 2.5), np.percentile(prc_aucs, 97.5)
    print("prc_auc_CI",prc_auc_CI)
    
    auc_CI = np.percentile(aucs, 2.5), np.percentile(aucs, 97.5)
    ##
    
    precision_CI = np.percentile(precisions, 2.5), np.percentile(precisions, 97.5)
    f1s_CI = np.percentile(f1s, 2.5), np.percentile(f1s, 97.5)
    acc_CI = np.percentile(acc, 2.5), np.percentile(acc, 97.5)
    return acc_mean,precisions_mean,f1s_mean,auc_mean,auc_CI,prc_auc_CI,precision_CI,f1s_CI,acc_CI


In [ ]:
###### performance criteria
precisions_mean_lst =[]
f1s_mean_lst,auc_mean_lst,acc_mean_lst = [],[],[]
precisions_CI_lst,f1s_CI_lst,acc_CI_lst = [],[],[],[]
auc_CI_lst,prcauc_CI_list = [],[]
for i in classifier_lst:
    acc_mean,precisions_mean,f1s_mean,auc_mean, auc_CI,
    prc_auc_CI,spc_CI,npv_CI,precision_CI,f1s_CI,acc_CI = make_baseline(data,i)
    acc_mean_lst.append(acc_mean)
    precisions_mean_lst.append(precisions_mean)
    f1s_mean_lst.append(f1s_mean)
    auc_mean_lst.append(auc_mean)   
    auc_CI_lst.append(auc_CI)
    prcauc_CI_list.append(prc_auc_CI)    
    precisions_CI_lst.append(precision_CI)
    f1s_CI_lst.append(f1s_CI)
    acc_CI_lst.append(acc_CI)
    
print("acc:",acc_mean_lst)
print("precision:",precisions_mean_lst)
print("f1_score:",f1s_mean_lst)
print("auc:",auc_mean_lst)
print("auc_CI:",auc_CI_lst)
all_para = {"accuracy":acc_mean_lst,"precision":precisions_mean_lst,
            "f1score":f1s_mean_lst,
            "aucscore":auc_mean_lst,            
            "accuracy_CI":acc_CI_lst,"precision_CI":precisions_CI_lst,
            "f1_score_CI":f1s_CI_lst,
            "auc_CI":auc_CI_lst,"prc_auc_CI":prcauc_CI_list}
result = DataFrame(all_para,index = ['LR', 'SVM', 'DT', 'XGboost', 'RF'])
print(result[result.columns[0:7]])

In [ ]:
#plot ROC curve

X,y=np.array(feature),np.array(target)
print(X.shape,y.shape)
cv = StratifiedKFold(n_splits=5,random_state=1)

from sklearn.metrics import roc_curve, auc,roc_auc_score
from scipy import interp
import warnings
warnings.filterwarnings("ignore")

def plot_ROC(model,model_name,color):
    
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    i = 0
    for train, test in cv.split(X, y):
        probas_ = model.fit(X[train], y[train]).predict_proba(X[test])
        fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1]) #,pos_label=1
        tprs.append(interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        i += 1
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    roc_auc_CI = np.percentile(aucs, 2.5), np.percentile(aucs, 97.5)
    print("roc_auc_CI",roc_auc_CI)
    plt.plot(mean_fpr, mean_tpr, color=color,
             label=r'{} (AUC = %0.2f, 95%%CI [%0.2f, %0.2f])'.format(model_name) % (mean_auc, roc_auc_CI[0], roc_auc_CI[1]),
             lw=2, alpha=.8)

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)

color_list = ['b','g','r','c','m']
fig=plt.gcf()
fig.set_size_inches(10,10)
for model,model_name,color in zip(classifier_lst,classifier_lst_name,color_list):
    plot_ROC(model,model_name,color)
    
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color=(0.6, 0.6, 0.6),label='Chance', alpha=.8)
plt.xlim([-0.0, 1.0])
plt.ylim([-0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
#plt.title('Receiver operating characteristic')
plt.legend(loc="lower right", fontsize='small')
fig.show()

plt.savefig("ROC.png", dpi = 1080)